**Describing the variables in the cancer data set**

In [2]:
 ### Run this cell before continuing.
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [3]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine <- read_csv2(url)

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.

Parsed with column specification:
cols(
  `fixed acidity` = col_number(),
  `volatile acidity` = col_character(),
  `citric acid` = col_character(),
  `residual sugar` = col_number(),
  chlorides = col_character(),
  `free sulfur dioxide` = col_number(),
  `total sulfur dioxide` = col_double(),
  density = col_character(),
  pH = col_number(),
  sulphates = col_character(),
  alcohol = col_number(),
  quality = col_double()
)

Warning message:
“2 parsing failures.
 row                  col               expected actual                                                                                         file
1296 total sulfur dioxide no trailing characters     .5 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
1297 total sulfur dioxide no trailing characters     .5 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
”

Clean and wrangle our data

In [4]:
colnames(wine) <- make.names(colnames(wine))

wine <- wine %>% 
  mutate(quality = as_factor(quality),
        volatile.acidity = as.double(volatile.acidity),
    citric.acid = as.double(citric.acid),
  chlorides = as.double(chlorides),
  density = as.double(density),
  sulphates = as.double(sulphates))
  

wine
nrow(wine)



fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,5
78,0.88,0.00,26,0.098,25,67,0.9968,32,0.68,98,5
78,0.76,0.04,23,0.092,15,54,0.9970,326,0.65,98,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
63,0.510,0.13,23,0.076,29,40,0.99574,342,0.75,11,6
59,0.645,0.12,2,0.075,32,44,0.99547,357,0.71,102,5
6,0.310,0.47,36,0.067,18,42,0.99549,339,0.66,11,6


[1] 1599

In [5]:
nrow(wine)

sum(is.na(wine))

newdata <- na.omit(wine)
nrow(newdata)


[1] 1599

[1] 2

[1] 1597

In [6]:
set.seed(15)

wine_split <- initial_split(wine, prop = 0.75, strata = quality)  
wine_train <- training(wine_split)
wine_test <- testing(wine_split)

wine_train
wine_test

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,5
78,0.76,0.04,23,0.092,15,54,0.9970,326,0.65,98,5
74,0.70,0.00,19,0.076,11,34,0.9978,351,0.56,94,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
63,0.510,0.13,23,0.076,29,40,0.99574,342,0.75,11,6
59,0.645,0.12,2,0.075,32,44,0.99547,357,0.71,102,5
6,0.310,0.47,36,0.067,18,42,0.99549,339,0.66,11,6


fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
78,0.88,0.00,26,0.098,25,67,0.9968,32,0.68,98,5
112,0.28,0.56,19,0.075,17,60,0.9980,316,0.58,98,6
67,0.58,0.08,18,0.097,15,65,0.9959,328,0.54,92,5
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
54,0.74,0.09,17,0.089,16,26,0.99402,367,0.56,116,6
62,0.60,0.08,2,0.090,32,44,0.99490,345,0.58,105,5
59,0.55,0.10,22,0.062,39,51,0.99512,352,0.76,112,6


In [7]:
num_obs <- nrow(wine)

wine_summary <- wine %>%
  group_by(quality) %>%
  summarize(
    count = n(),
    percentage = (n()/num_obs) * 100
  )

wine_summary

`summarise()` ungrouping output (override with `.groups` argument)



quality,count,percentage
<fct>,<int>,<dbl>
3,10,0.6253909
4,53,3.3145716
5,681,42.5891182
6,638,39.8999375
7,199,12.4452783
8,18,1.1257036


In [8]:
num_obs_train <- nrow(wine_train)

wine_summary_train <- wine_train %>%
  group_by(quality) %>%
  summarize(
    count = n(),
    percentage = (n()/num_obs) * 100,
    sulphates_mean = mean(sulphates, na.rm = TRUE),
    pH_mean = mean(pH, na.rm = TRUE),
    total.sulfur.dioxide_mean = mean(total.sulfur.dioxide, na.rm = TRUE),
    
  )

wine_summary_train


`summarise()` ungrouping output (override with `.groups` argument)



quality,count,percentage,sulphates_mean,pH_mean,total.sulfur.dioxide_mean
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
3,7,0.4377736,0.5685714,300.2857,24.57143
4,38,2.3764853,0.6086842,282.3421,37.31579
5,512,32.0200125,0.6163086,298.9609,57.13922
6,480,30.0187617,0.6729167,299.5958,40.51042
7,148,9.2557849,0.7410135,300.3514,36.10135
8,15,0.9380863,0.7480000,289.3333,34.60000


In [9]:
wine <- wine %>% 
  mutate(sulphates_mean = mean(sulphates))
mean(sulphates)
wine

ERROR: Error in mean(sulphates): object 'sulphates' not found
